## FORWARD SOLVER: game(p,E,s, b, C) ==> (x, v)

In [10]:
using NLsolve
using LinearAlgebra

"""
Compute entropy-regularized Nash mixed strategies by Newton's method
Inputs:
- pa, struct of parameters containing:
    - p, number of players
    - E, incidence matrix, where (n, m) = size(E)
    - s, source-sink vector
    - b, where b_i is the nominal price of each link for all i ∈ [p] players
    - C, additional cost adjustment introduced by other players
- λ, entropy weight

Returns:
- x: mixed eq strategies for each player on space of links
- v: value vector/function associated to all nodes
"""

function solve_entropy_routing(pa, λ)
    # println("calling solve_entropy_routing...")

    function nash!(F, xi)
        # x = xi[1:pa.p*pa.m]
        # v = xi[pa.p*pa.m+1:end]
        # F[1:pa.p*pa.m] = x - exp.(1/λ * (kron(I(pa.p), pa.E') * v - pa.b - pa.C * x) - ones(pa.p*pa.m,1))
        # F[pa.p*pa.m+1:end] = pa.s - kron(I(pa.p), pa.E) * x

        for i in 1:pa.p*pa.m
            F[i] = xi[1:pa.p*pa.m][i] - exp(1/λ * ((kron(I(pa.p), pa.E') * xi[pa.p*pa.m+1:end])[i] - pa.b[i] - (pa.C * xi[1:pa.p*pa.m])[i]) - ones(pa.p*pa.m,1)[i])

        end
        for j in 1:pa.p*pa.n
            F[pa.p*pa.m+j] = pa.s[j] - (kron(I(pa.p), pa.E) * xi[1:pa.p*pa.m])[j]
        end
    end

    x0 = 0.5.*ones(pa.p*pa.m, 1)
    v0 = 0.5.*ones(pa.p*pa.n, 1)
    sol = nlsolve(nash!, [x0; v0], autodiff = :forward, show_trace=false, ftol=1e-8, iterations=1000)

    (;x = sol.zero[1:pa.p*pa.m],
      v = sol.zero[pa.p*pa.m+1:end])
end

solve_entropy_routing (generic function with 1 method)

## BACKWARD SOLVER: (p,E,s, x_hat) ==> (b, C)

In [3]:
using Plots, GraphPlot, Colors
using Graphs, GraphRecipes
using ArgParse
using JLD2
using IterativeSolvers

# helper function: plot unlabeled directed graph - GRID_GRAPH
function plot_unlabeled(game_name, g, row_num)
    # generating edgelabel_dict
    edgelabel_dict = Dict{Tuple{Int64, Int64}, String}()
    for i in 1:ne(g)
        col = -Matrix(incidence_matrix(g))[:,i]
        edgelabel_dict[(findall(x->x==1, col)[1], findall(x->x==-1, col)[1])] =string(i)
    end
    # plotting directed graph with node and link labelings
    δ=0.01
    locs_x = collect(Iterators.flatten([i%2==1 ? [1:row_num...] : [1:row_num...].+δ for i in 1:row_num]))
    locs_y = collect(Iterators.flatten([repeat([i],row_num) for i in row_num:-1:1]))
    graphplot(g,
              names=1:nv(g), x=2*locs_x, y=1.5*locs_y, nodeshape=:circle, nodesize=0.8,
              edgelabel=edgelabel_dict, curvature_scalar=0.2)
    savefig("($game_name$row_num)_unlabeled.png")
end

# helper function: projection onto D
function proj_D(C, ρ, pa)
    C1 = 0.5 * (C - C')
    for n in 1:pa.p
        C1[pa.m*(n-1)+1:pa.m*n, pa.m*(n-1)+1:pa.m*n] = zeros(pa.m, pa.m)
    end
    
    C2 = 0.5 * (C + C')
    s = eigvals(C2)
    U = eigvecs(C2)
    
    A = real(C1 + U * diagm(vec(max.(s,zeros(length(s))))) * U')
    return ρ / max.(ρ, norm(A)) * A
end

proj_D (generic function with 1 method)

In [11]:
"""
Approximate projected gradient method (measuring function ψ(x)=1/2*||x-x̂||^2)
Inputs:
- p, number of players
- E, incidence matrix
- s, source-sink vector
- x̂, desired Nash solution
- λ, entropy weight
- α, step size
- ϵ, stopping tolerance
- ρ, modification allowance
- max_iter

Returns:
- x: mixed eq strategies for each player on space of links
- b
- C
"""

function approx_proj_grad(p, E, s, x̂, λ, α, ϵ, ρ, max_iter, x_init, b_init, C_init)
    n, m = size(E)
    
    # initiate b, b_plus, C, C_plus AND x
    b = zeros(p*m)
    b_plus = b_init
    C = 0.1*I(p*m)
    C_plus = C_init
    x = x_init

    v = zeros(p*n)
    
    # data output placeholders
    ψ_vals = Float64[]
    lambda_vals = Float64[]

    println("Starting approx proj grad...")

    for i in 1:max_iter
        # if max(norm(b-b_plus), norm(C-C_plus)) <= ϵ || 0.5 * norm(x-x̂)^2 <= ϵ
        # if max(norm(b-b_plus), norm(C-C_plus)) <= ϵ
        # if 0.5 * norm(x-x̂)^2 <= ϵ
        # if vio(b, C, x, v) <= ϵ
        #     println("Converged.")
        #     break
        # elseif i == max_iter
        #     println("Reached max_iter of $max_iter, break.")
        #     break
        # else
            
            # update b, C
            b = b_plus
            C = C_plus
        
            # create parameters block and solve for x
            function pa()
                pa_p = p
                pa_E = E
                pa_n, pa_m = n, m
                pa_s = s
                pa_b = b
                pa_C = C
                (; p=pa_p, E=pa_E, n=pa_n, m=pa_m, s=pa_s, b=pa_b, C=pa_C)
            end
            x, v = solve_entropy_routing(pa(), λ)

            # measure ψ(x) val
            push!(ψ_vals, 0.5 * norm(x-x̂)^2)

            # record current λ value
            push!(lambda_vals, λ)
            
            # compute D, J 
            D = diagm(vec(exp.(1/λ * (kron(I(p), E')*v-b-C*x) - ones(p*m, 1)))) # dim: 18x18
            J = [I(p*m)+1/λ*D*C 1/λ*D*kron(I(p), E'); kron(-I(p), E) zeros(p*n, p*n)] # dim: 27x27

            # compute ∇ψ_x
            ∇ψ_x = x - x̂ # 18x1

            # compute ∇̂ψ_b, ∇̂ψ_C
            # ∇̂ψ_b = - 1/λ * [D' zeros(p*m, p*n)] * pinv(J)' * [∇ψ_x; zeros(p*n)]
            ∇̂ψ_C = - 1/λ * [D' zeros(p*m, p*n)] * pinv(J)' * [∇ψ_x; zeros(p*n)] * x' # 18x1

            # update b_plus, C_plus
            # b_plus = proj_B(b - α * ∇̂ψ_b, 0.1)
            C_plus = proj_D(C - α * ∇̂ψ_C, ρ, pa())
        # end
    end

    (;x = x,
      v = v,
      b = b,
      C = C,
      ψ_vals = ψ_vals,
      lambda_vals = lambda_vals
)

end

approx_proj_grad (generic function with 1 method)

## RUNNING

### create and instantiate routing games

In [6]:
function grid_graph3_2_players()
    game_name = "grid_graph3_2_players"

    row_num = 3
    col_num = 3
    g = SimpleDiGraph(row_num * col_num)
    for i in 1:row_num
        for j in 1:col_num-1
        add_edge!(g, row_num*(i-1)+j, row_num*(i-1)+j+1)
        add_edge!(g, row_num*(i-1)+j+1, row_num*(i-1)+j)
        end
    end
    for j in 1:col_num
        for i in 1:row_num-1
            add_edge!(g, row_num*(i-1)+j, row_num*(i-1)+j+row_num)
            add_edge!(g, row_num*(i-1)+j+row_num, row_num*(i-1)+j)
        end
    end

    p = 2
    E = -Matrix(incidence_matrix(g))
    n, m = size(E)

    s_p1 = zeros(9); s_p1[4] = 1; s_p1[6] = -1;
    s_p2 = -s_p1
    s = [s_p1; s_p2]

    p1 = zeros(24); p1[8] = p1[1] = p1[4] = p1[7] = 1;
    p2 = zeros(24); p2[17] = p2[24] = p2[21] = p2[18] = 1;
    x̂ = vec([p1; p2]) # dim: 24*2

    # plot_unlabeled(game_name, g, row_num)
    (;game_name=game_name, g=g, p=p, E=E, n=n, m=m, s=s, x̂=x̂)
end


grid_graph3_2_players (generic function with 1 method)

### running and saving result

In [22]:
# instantiate a routing game (p, E, s) with desired Nash sol x̂
game_name, g, p, E, n, m, s, x̂ = grid_graph3_2_players()


# assign parameters
λ = 0.01
α = 0.01
ϵ = 0.01
ρ = 0.15
max_iter = 10

n, m = size(E)
x_init = zeros(p*m)
b_init = 0.1*ones(p*m)
C_init = zeros(p*m, p*m)

# calling method
println("----------- $(game_name)_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ) -----------")
x, x_init, b, C, ψ_vals = approx_proj_grad(p, E, s, x̂, λ, α, ϵ, ρ, max_iter, x_init, b_init, C_init)


""" SAVING RESULT """
# create an individual folder under results/
dir = "results/$(game_name)/$(game_name)_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ)"; mkpath(dir) # mkdir if not exists
println("saving plot and output to '$dir'")

# plot and save
plot(ψ_vals, xlabel="iter", ylabel="ψ(x)"); savefig("$dir/ψ_vals_plots_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ).png")
@show ψ_vals

# save outputs
@save "$dir/output.jld2" x x_init b C ψ_vals
# open("$dir/output.jl", "w") do output_file
#     write(output_file, "# ------------ λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ) ------------ \n \n")
    
#     write(output_file, "x = ") # writes x =
#     show(output_file, x) # writes the content of x
#     write(output_file, "; \n \n")
    
#     write(output_file, "x_init = ")
#     show(output_file, x_init) 
#     write(output_file, "; \n \n")

#     write(output_file, "b = ")
#     show(output_file, b) 
#     write(output_file, "; \n \n")

#     write(output_file, "C = ")
#     show(output_file, C) 
#     write(output_file, "; \n \n")

#     write(output_file, "ψ_val[end] = ")
#     show(output_file, ψ_vals[end]) 
#     write(output_file, "; \n \n")
# end
println("--------------------------------")

----------- grid_graph3_2_players_λ=(0.01)_α=(0.01)_ϵ=(0.01)_ρ=(0.15) -----------
Starting approx proj grad...
saving plot and output to 'results/grid_graph3_2_players/grid_graph3_2_players_λ=(0.01)_α=(0.01)_ϵ=(0.01)_ρ=(0.15)'
ψ_vals = [5.935143931693041, 5.108861500181219, 1.93522248166304, 1.2266212883690253, 0.8661378724383382, 0.7442967089318397, 0.7213165144525591, 0.7187735101126814, 0.718624555153909, 0.7197629783748426]
--------------------------------


### PLOTTING RHO GRAPH

In [24]:
using JLD2, CSV, DataFrames

""" grid_graph3_2_players """
rho_list = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5] 

""" grid_graph3_4_players """
# rho_list = [0.01, 0.03, 0.05, 0.1, 0.13, 0.15, 0.17, 0.18, 0.2, 0.3, 0.5, 0.6] 

# alpha_list = [0.01, 0.01, 0.01, 0.01, 0.005, 0.01, 0.005, 0.01, 0.01, 0.01, 0.01, 0.003]

# epsilon_list = 0.01 * ones(length(rho_list)); 
# epsilon_list[findall(x->x==0.13, rho_list)[1]] = 0.02; 
# epsilon_list[findall(x->x==0.15, rho_list)[1]] = 0.05


""" grid_graph5_2_players """
# rho_list = [0.01, 0.05, 0.08, 0.1, 0.13, 0.14, 0.15, 0.2, 0.3, 0.5]

# alpha_list = 0.01 * ones(length(rho_list)); 
# alpha_list[findall(x->x==0.08, rho_list)[1]] = 0.005; 
# alpha_list[findall(x->x==0.1, rho_list)[1]] = 0.005
# alpha_list[findall(x->x==0.15, rho_list)[1]] = 0.005
# alpha_list[findall(x->x==0.13, rho_list)[1]] = 0.001
# alpha_list[findall(x->x==0.14, rho_list)[1]] = 0.001

# epsilon_list = 0.01 * ones(length(rho_list)); 
# epsilon_list[findall(x->x==0.08, rho_list)[1]] = 0.05; 
# epsilon_list[findall(x->x==0.1, rho_list)[1]] = 0.08
# epsilon_list[findall(x->x==0.15, rho_list)[1]] = 0.05
# epsilon_list[findall(x->x==0.13, rho_list)[1]] = 0.01
# epsilon_list[findall(x->x==0.14, rho_list)[1]] = 0.005

""" grid_graph5_4_players """
# rho_list = [0.01, 0.05, 0.1, 0.3, 0.5]
# # 0.08, 0.1, 0.2, 0.3, 0.5, 0.7] 

# alpha_list = 0.01 * ones(length(rho_list));

# epsilon_list = 0.01 * ones(length(rho_list)); 
# epsilon_list[findall(x->x==0.1, rho_list)[1]] = 0.1; 
# epsilon_list[findall(x->x==0.5, rho_list)[1]] = 0.1; 
# epsilon_list[findall(x->x==0.3, rho_list)[1]] = 0.1; 

""" PLOTTING RHO GRAPH"""
# collect psi_end_list
psi_end_list = Float64[]
for i in 1:length(rho_list)
    ρ = rho_list[i]
    # α = alpha_list[i]
    # ϵ = epsilon_list[i]
    @load "results/$(game_name)/$(game_name)_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ)/output.jld2" ψ_vals
    println("ρ=$ρ", "   ", "ψ_vals[end]=$(ψ_vals[end])")
    push!(psi_end_list, ψ_vals[end])
end

# plot and save rho graph in results/
println("saving rho paramter search plot in 'results/$(game_name)_rho_parameter_search.png'")
plot(rho_list, psi_end_list, xlabel="ρ", ylabel="ψ(x) convergence value", markershape = :circle)
savefig("results/$(game_name)/$(game_name)_rho_parameter_search.png")

df = DataFrame(rho = rho_list,
                psi_convergence_val = psi_end_list)
CSV.write("3x3_2p_rho_plot.csv", df)

ρ=0.01   ψ_vals[end]=5.873020585843631
ρ=0.05   ψ_vals[end]=4.780063891663585
ρ=0.1   ψ_vals[end]=2.45060439622661
ρ=0.15   ψ_vals[end]=0.7197629783748426
ρ=0.2   ψ_vals[end]=0.1871533726527597
ρ=0.3   ψ_vals[end]=0.002840208913223733
ρ=0.5   ψ_vals[end]=0.0027633556521115808
saving rho paramter search plot in 'results/grid_graph3_2_players_rho_parameter_search.png'


"3x3_2p_rho_plot.csv"

### homotopy

##### parameter choice

In [ ]:
""" RUNNING EXP: PARAMETER CHOICE """
# instantiate a routing game (p, E, s) with desired Nash sol x̂
game_name, g, p, E, s, x̂ = grid_graph3_4_players()

# assign parameters
α = 0.01
ϵ = 1e-3
ρ = 0.5
max_iter = 10

@show λ_list = [1.0/(2^i) for i in 5:10]
# λ_list = [0.1, 0.01, 0.005, 0.003, 0.002]

# calling method
println("----------- $(game_name)_ρ=($ρ)_λ_list=($λ_list)_α=($α)_ϵ=($ϵ) -----------")
x, b, C, ψ_vals_list, violation_metrics_list, lambda_vals_list, v, ∇̂ψ_C_norm_list, D_norm_list, J_norm_list, pinv_J_norm_list, F_norm_list = homotopy_exp_parameter_choice(p, E, s, x̂, λ_list, α, ϵ, ρ, max_iter)

# """ SAVING RESULT """
# create an individual folder under homotopy_results/
dir = "homotopy_results/$(game_name)"; mkpath(dir) # mkdir if not exists

# save data to folder
println("saving result to '$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2'")
@save "$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list v ∇̂ψ_C_norm_list D_norm_list J_norm_list pinv_J_norm_list F_norm_list
println("--------------------------------")

In [ ]:
# @load "$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list v ∇̂ψ_C_norm_list D_norm_list J_norm_list pinv_J_norm_list F_norm_list
dir = dir = "homotopy_results/grid_graph3_4_players"
result_dir = "homotopy_results/grid_graph3_4_players/grid_graph3_4_players_λ_list=([0.03125, 0.015625, 0.0078125, 0.00390625, 0.001953125, 0.0009765625])_homotopy.jld2"
@load result_dir ρ λ_list α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list v ∇̂ψ_C_norm_list D_norm_list J_norm_list pinv_J_norm_list F_norm_list

# plot single axis (log-scaled)
plot(lambda_vals_list, yscale=:log10, c=:green, label="λ", leg=:bottomleft, xlabel="iter", ylabel="log10 scaled")
plot!(ψ_vals_list, yscale=:log10, c=:blue, linestyle=:dash, label="ψ(x)", leg=:bottomleft)
plot!(violation_metrics_list, yscale=:log10, c=:red, linestyle=:dash, label="violation metric", leg=:bottomleft)
hline!([1e-3], label="1e-3", c=:grey)
plot!(title="ρ=($ρ), α=($α), ϵ=($ϵ)")
savefig("$dir/$(game_name)_λ_list=($λ_list)_homotopy.png")

In [ ]:
@show F_norm_list
# plot(∇̂ψ_C_norm_list, label="∇̂ψ_C_norm_list", yscale=:log10, leg=:bottomleft)
# plot!(D_norm_list, label="D_norm_list", yscale=:log10)
# plot!(J_norm_list, label="J_norm_list", yscale=:log10)
# plot!(pinv_J_norm_list, label="pinv_J_norm_list", yscale=:log10)
plot(F_norm_list, label="F_norm_list", yscale=:log10)
# hline!([1e-3], label="1e-3", c=:grey)
savefig("$dir/$(game_name)_λ_list=($λ_list)_∇̂ψ_C_norm_list.png")

##### lambda step

In [ ]:
""" RUNNING EXP: LAMBDA STEP """
# instantiate a routing game (p, E, s) with desired Nash sol x̂
game_name, g, p, E, s, x̂ = grid_graph3_4_players()

# assign parameters
α = 0.01
ϵ = 1e-3
ρ = 0.5
max_iter = 10

λ_init = 0.4/16
λ_step = 2
λ_threshold = 1e-3

# calling method
println("----------- $(game_name)_ρ=($ρ)_λ_init=($λ_init)_λ_step=($λ_step)_α=($α)_ϵ=($ϵ) -----------")
x, b, C, ψ_vals_list, violation_metrics_list, lambda_vals_list, v = homotopy_exp_step(p, E, s, x̂, λ_init, λ_step, λ_threshold, α, ϵ, ρ, max_iter)

""" SAVING RESULT """
# create an individual folder under homotopy_results/
dir = "homotopy_results/$(game_name)"; mkpath(dir) # mkdir if not exists

# save data to folder
println("saving result to '$dir/$(game_name)_ρ=($ρ)_λ_init=($λ_init)_λ_step=($λ_step)_α=($α)_ϵ=($ϵ)_homotopy.jld2'")
@save "$dir/$(game_name)_ρ=($ρ)_λ_init=($λ_init)_λ_step=($λ_step)_α=($α)_ϵ=($ϵ)_homotopy.jld2" ρ λ_init λ_step λ_threshold α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list v
println("--------------------------------")

In [ ]:
@load "$dir/$(game_name)_ρ=($ρ)_λ_init=($λ_init)_λ_step=($λ_step)_α=($α)_ϵ=($ϵ)_homotopy.jld2" ρ λ_init λ_step λ_threshold α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list

 # plot single axis (log-scaled)
 plot(lambda_vals_list, yscale=:log10, c=:green, label="λ", leg=:bottomleft, xlabel="iter", ylabel="log10 scaled")
 plot!(ψ_vals_list, yscale=:log10, c=:blue, linestyle=:dash, label="ψ(x)", leg=:bottomleft)
 plot!(violation_metrics_list, yscale=:log10, c=:red, linestyle=:dash, label="violation metric", leg=:bottomleft)
 plot!
 savefig("$dir/$(game_name)_ρ=($ρ)_λ_init=($λ_init)_λ_step=($λ_step)_α=($α)_ϵ=($ϵ)_homotopy.png")

 @show violation_metrics_list[end]
 
# plot dual axis
# pyplot()
# plot(ψ_vals, linestyle=:dash, xlabel="iter", label="ψ(x)", leg=:bottomleft, box=:true, right_margin=15Plots.mm)
# plt = twinx()
# plot!(plt, violation_metrics, linestyle=:dash, label="violation metric", leg=:topright, c=:red, box=:true)
# plot!(plt, lambda_vals, yscale=:log, label="lambda", leg=:topright, c=:green)
# savefig("$dir/$(game_name)_multi_plots_α=($α)_ϵ=($ϵ)_ρ=($ρ).png")
# println("--------------------------------")

@show x

In [ ]:
function homotopy_exp_parameter_choice(p, E, s, x̂, λ_list, α, ϵ, ρ, max_iter)
    n, m = size(E)

    # output placeholder
    x = zeros(p*m)
    b = 0.1*ones(p*m)
    C = zeros(p*m, p*m)
    v = zeros(p*n)
    
    ψ_vals_list = Float64[]
    violation_metrics_list = Float64[]
    lambda_vals_list = Float64[]
    ∇̂ψ_C_norm_list = Float64[]
    D_norm_list = Float64[]
    J_norm_list = Float64[]
    pinv_J_norm_list = Float64[]
    F_norm_list = Float64[]

    for λ in λ_list
        println("λ=$λ, α=$α")
        x, b, C, ψ_vals, violation_metrics, lambda_vals, v, ∇̂ψ_C_norm, D_norm, J_norm, pinv_J_norm, F_norm = approx_proj_grad(p, E, s, x̂, λ, α, ϵ, ρ, max_iter, x, b, C)
        append!(ψ_vals_list, ψ_vals)
        append!(violation_metrics_list, violation_metrics)
        append!(lambda_vals_list, lambda_vals)
        append!(∇̂ψ_C_norm_list, ∇̂ψ_C_norm)
        append!(D_norm_list, D_norm)
        append!(J_norm_list, J_norm)
        append!(pinv_J_norm_list, pinv_J_norm)
        append!(F_norm_list, F_norm)
    end
    
    (;x = x,
      b = b,
      C = C,
      ψ_vals_list = ψ_vals_list,
      violation_metrics_list = violation_metrics_list,
      lambda_vals_list = lambda_vals_list,
      v = v,
      ∇̂ψ_C_norm_list = ∇̂ψ_C_norm_list,
      D_norm_list = D_norm_list,
      J_norm_list = J_norm_list,
      pinv_J_norm_list = pinv_J_norm_list,
      F_norm_list = F_norm_list)
end

In [ ]:
function homotopy_exp_step(p, E, s, x̂, λ_init, λ_step, λ_threshold, α, ϵ, ρ, max_iter)
    n, m = size(E)

    # init λ
    λ = λ_init
    
    # output placeholder
    x = zeros(p*m)
    b = 0.1*ones(p*m)
    C = zeros(p*m, p*m)
    ψ_vals_list = Float64[]
    violation_metrics_list = Float64[]
    lambda_vals_list = Float64[]

    v = zeros(p*n)

    while λ > λ_threshold
        println("λ=$λ, α=$α")
        x, b, C, ψ_vals, violation_metrics, lambda_vals, v = approx_proj_grad(p, E, s, x̂, λ, α, ϵ, ρ, max_iter, x, b, C)
        append!(ψ_vals_list, ψ_vals)
        append!(violation_metrics_list, violation_metrics)
        append!(lambda_vals_list, lambda_vals)
        λ /= λ_step
    end
    
    (;x = x,
      b = b,
      C = C,
      ψ_vals_list = ψ_vals_list,
      violation_metrics_list = violation_metrics_list,
      lambda_vals_list = lambda_vals_list,
      v = v)
end